In [1]:
import pandas as pd
from moviepy.editor import VideoFileClip
import os

C:\Users\hennecol\AppData\Local\Temp\ipykernel_9460\1838224339.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
from moviepy.editor import VideoFileClip

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [3]:
# Define project information
PROJECT_ID = "lunar-airport-418323"  # @param {type:"string"}
LOCATION = "asia-southeast1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

In [5]:
# Define GenerationConfig with desired parameters
generation_config = GenerationConfig(
    temperature=0.0,  # Adjust temperature between 0.0 (deterministic) and 1.0 (more random)
    #top_p=0.3,  # Adjust top_p between 0.0 (likely tokens) and 1.0 (considers all tokens)
    #top_k= 32
)

In [6]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

In [7]:
#Define uri for the full video
video_name = "part 2 - 3 video 1_Trim" #without the .mp4 extension
video_bucket_nb = 2 #1 or 2

video_uri = f"gs://datavid{video_bucket_nb}/{video_name}.mp4"
video_uri


'gs://datavid2/part 2 - 3 video 1_Trim.mp4'

In [8]:
#Test with full video
prompt = """
This video is a first person view from inside a moving car, in a country where people drive on the left. Answer the following questions using the video only:
You are the driver of the camera's car, can you spot a situation that will end in an accident where another element is in the same trajectory as your car? If yes:
Say yes, describe the situation and give the elements involved.
If no:
Say no and describe what is shown in the video.
"""

"""
This video is a first person view from inside a moving car, in a country where people drive on the left. If there is a situation where an accident is about to occur, the video is cut just before the accident occurs, so the accident situation is just before the end of the video.
Answer the following questions using the video only:
You are the driver of the camera's car, can you spot a situation where an accident is about to occur with your car? If yes:
Describe the situation and give the elements involved and where they come from in relation to your car.
"""

'''This video is taken from inside a car, from the front passenger seat, in a country where people drive on the left. If there is a situation where an accident is about to occur, the video is cut just before the accident occurs, so the accident situation is just before the end of the video.
Answer the following questions using the video only:
Can you spot a situation where an accident is about to occur in the final moments? If yes:
Describe the situation and give the elements involved.'''

'''This video is a first person view from inside a moving car, in a country where people drive on the left. If there is a situation where an accident is about to occur, the video is cut just before the accident occurs, so the accident situation is just before the end of the video.
Answer the following questions using the video only:
Can you spot a situation where an accident is about to occur with the camera? If yes:
Describe the situation, give the elements involved and where they come from in relation to the camera.'''


video = Part.from_uri(video_uri, mime_type="video/mp4")
contents = [video, prompt]

responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)

for response in responses:
    print(response.text, end="")

 Yes, there is a situation that will end in an accident. A white car is driving in the same trajectory as the camera's car, but in the opposite direction. The white car is on the wrong side of the road and is driving towards the camera's car. If the two cars continue on their current paths, they will collide head-on.

Now, we cut the video and keep only the part after risk_time.

In [ ]:
#Code to trim the video after risk_time
# Define the paths
csv_path = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/2 analysis and first preprocessing/liste_videos_pour_script - Copy.csv"  # Adjust this to the correct path
video_source_folder = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/1 experiment/final/datavid1"
video_dest_folder = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/1 experiment/final/datavid1/videos_cuts"

# Ensure the destination folder exists
if not os.path.exists(video_dest_folder):
    os.makedirs(video_dest_folder)

# Read the CSV file
df = pd.read_csv(csv_path)

# Loop through each row in the dataframe
for index, row in df.iterrows():
    video_name = row['nom']
    risk_time = row['risk_time']

    # Check if risk_time is a number (ignoring NaN or non-numeric values)
    if pd.notna(risk_time) and isinstance(risk_time, (int, float, str)):
        try:
            risk_time = float(risk_time)  # Ensure risk_time is in seconds
            # Define the source and destination paths for the video
            source_path = video_source_folder + "/" + video_name
            dest_path = video_dest_folder + "/" + f"cut_{video_name}"
            print(source_path, dest_path)

            # Load the video
            video = VideoFileClip(source_path)

            # Cut the video from risk_time to the end
            cut_video = video.subclip(risk_time, video.duration)

            # Save the cut video with a video format extension
            cut_video.write_videofile(dest_path, codec="libx264", audio_codec="aac")

            # Close the video file to free up resources
            video.close()
            cut_video.close()

            print(f"Video {video_name} cut and saved successfully.")
        except Exception as e:
            print(f"Error processing video {video_name}: {e}")

In [12]:
#Define uri for the trimmed video
video_cut_name = "12 crashes 4" #without the .mp4 extension
video_bucket_nb = 1 #1 or 2

video_cut_uri = f"gs://datavid{video_bucket_nb}/{video_cut_name}.mp4"
video_cut_uri


'gs://datavid1/12 crashes 4.mp4'

In [13]:
#Test with trimmed video
prompt = """
This video is a first person view from inside a moving car, in a country where people drive on the left.
You are the driver of the camera's car, can you spot a situation where an accident could happen with your car? If yes:
Describe the situation, give the time of the video it appears (in seconds from the beginning) and give the elements involved.
"""
#This video is taken from inside a car, from the front passenger seat, in a country where people drive on the left. This video shows a situation where an accident is about to occur. Can you describe the situation shown in the video ?

video = Part.from_uri(video_cut_uri, mime_type="video/mp4")
contents = [video, prompt]

responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)

for response in responses:
    print(response.text, end="")

 There is a situation where an accident could happen with the car of the person filming.
At 0:19 seconds into the video, a silver car is seen approaching from the right, on the wrong side of the road. It is likely that this car will collide with the car of the person filming, as it is not possible for both cars to pass each other on the narrow road.

Now, we generate a description of every videos in the bucket folder.

In [32]:
csv_path = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/2 analysis and first preprocessing/liste_videos_pour_script_and_infos2.csv"  # Adjust this to the correct path
video_bucket_nb = 1 #1 or 2

prompt = """This video is a first person view from inside a moving car, in a country where people drive on the left. Answer the following questions using the video only:
You are the driver of the camera's car, can you spot a situation that will end in an accident where another element is in the same trajectory as your car? If yes:
Say yes, describe the situation and give the elements involved.
If no:
Say no and describe what is shown in the video.""" #prompt of the column description2

df = pd.read_csv(csv_path)

row = df.head(1)
video_uri = f"gs://datavid{video_bucket_nb}/{video_name}"
video = Part.from_uri(video_uri, mime_type="video/mp4")
contents = [video, prompt]
responses = [response.text for response in multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)]


InvalidArgument: 400 Request contains an invalid argument.

In [39]:
## Code to generate descriptions of the videos and store them in a .csv file

csv_path = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/2 analysis and first preprocessing/liste_videos_pour_script_and_infos2.csv"  # Adjust this to the correct path
video_bucket_nb = 1 #1 or 2

prompt = """This video is a first person view from inside a moving car, in a country where people drive on the left. Answer the following questions using the video only:
You are the driver of the camera's car, can you spot a situation that will end in an accident where another element is in the same trajectory as your car? If yes:
Describe the situation and give the elements involved.
If no:
Say no and describe what is shown in the video.""" #prompt of the column description2


#Prompt of the column "description"
# """
# This video is a first person view from inside a moving car, in a country where people drive on the left. If there is a situation where an accident is about to occur, the video is cut just before the accident occurs, so the accident situation is just before the end of the video.
# Answer the following questions using the video only:
# You are the driver of the camera's car, can you spot a situation where an accident is about to occur with your car? If yes:
# Describe the situation and give the elements involved.
# """

df = pd.read_csv(csv_path)

for index, row in df.iterrows():
    video_name = row['nom']
    video_uri = f"gs://datavid{video_bucket_nb}/{video_name}"
    video = Part.from_uri(video_uri, mime_type="video/mp4")
    contents = [video, prompt]
    responses = [response.text for response in multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)]
    # responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)
    # for response in responses:
    #     print(response.text, end="")

    response_text = ''.join(responses)
    df.at[index, 'description3'] = response_text # Update the 'description2' column for the current row
    print(f'Line {index} processed')

df.to_csv(csv_path, index=False)


Line 0 processed
Line 1 processed
Line 2 processed
Line 3 processed
Line 4 processed
Line 5 processed
Line 6 processed
Line 7 processed
Line 8 processed
Line 9 processed
Line 10 processed
Line 11 processed
Line 12 processed
Line 13 processed
Line 14 processed
Line 15 processed
Line 16 processed
Line 17 processed
Line 18 processed
Line 19 processed
Line 20 processed
Line 21 processed
Line 22 processed
Line 23 processed
Line 24 processed
Line 25 processed
Line 26 processed
Line 27 processed
Line 28 processed
Line 29 processed
Line 30 processed
Line 31 processed
Line 32 processed
Line 33 processed
Line 34 processed


New method : extract two frames from the video at risk_time and (risk_time + end_time)/2

In [187]:
## Function to extract frames from the video:
import cv2
import math

def extract_frames(video_path, video_name, time, output_folder):
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)
    
    # Vérifier si la vidéo a été chargée correctement
    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la vidéo.")
        return
    
    # Obtenir le nombre total de frames et le fps (frames par seconde)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Calculer les numéros des frames à extraire
    frame_num_time_sec = int(time * fps)  # Frame à la seconde time
    half_way_frame = int((((total_frames / fps) + time)/ 2) * fps)  # Frame à la moitié du temps entre time et la fin
    
    # Définir les moments d'extraction
    moments_to_extract = [frame_num_time_sec, half_way_frame]
    
    for count, moment in enumerate(moments_to_extract, start=1):
        if moment >= total_frames:
            print(f"Le moment d'extraction {moment/fps:.2f} secondes dépasse la durée de la vidéo.")
            continue
        # Se positionner sur la frame à extraire
        cap.set(cv2.CAP_PROP_POS_FRAMES, moment)
        
        # Lire la frame
        success, frame = cap.read()
        if success:
            # Sauvegarder la frame extraite
            output_path = output_folder + "/" + video_name.split(".")[0] + f"frame_{count}.jpg"
            cv2.imwrite(output_path, frame)
            print(f"Frame à {moment/fps:.2f} secondes sauvegardée.")
        else:
            print(f"Erreur lors de l'extraction de la frame à {moment/fps:.2f} secondes.")
    
    # Libérer le capture vidéo
    cap.release()

# Remplacez 'chemin_vers_votre_video.mp4' par le chemin réel de votre vidéo
video_path = 'C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/1 experiment/final/datavid1/1 weekly - 3.mp4'
video_name = "1 weekly - 3.mp4"
time = 7.6
output_folder = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/1 experiment/final/datavid1/video_frames"
extract_frames(video_path,video_name, time, output_folder)

Frame à 7.60 secondes sauvegardée.
Frame à 8.03 secondes sauvegardée.


In [188]:
# Extract frames from all videos of a folder
csv_path = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/2 analysis and first preprocessing/liste_videos_pour_script_and_infos.csv"  # Adjust this to the correct path
video_source_folder = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/1 experiment/final/datavid1"
video_dest_folder = "C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/1 experiment/final/datavid1/video_frames"

# Ensure the destination folder exists
if not os.path.exists(video_dest_folder):
    os.makedirs(video_dest_folder)

# Read the CSV file
df = pd.read_csv(csv_path)

# Loop through each row in the dataframe
for index, row in df.iterrows():
    video_name = row['nom']
    risk_time = row['risk_time']

        # Check if risk_time is a number (ignoring NaN or non-numeric values)
    if pd.notna(risk_time) and isinstance(risk_time, (int, float, str)):
        try:
            risk_time = float(risk_time)  # Ensure risk_time is in seconds
            # Define the source and destination paths for the video
            video_path = video_source_folder + "/" + video_name
            extract_frames(video_path, video_name, risk_time, video_dest_folder)
        except Exception as e:
            print(f"Error processing video {video_name}: {e}")



Error processing video 15_low1_1.mp4: could not convert string to float: '-'
Error processing video 15_low3_2.mp4: could not convert string to float: '-'
Frame à 3.50 secondes sauvegardée.
Frame à 3.93 secondes sauvegardée.
Error processing video 11 controls 6.mp4: could not convert string to float: '-'
Frame à 10.20 secondes sauvegardée.
Frame à 12.67 secondes sauvegardée.
Frame à 3.70 secondes sauvegardée.
Frame à 4.50 secondes sauvegardée.
Frame à 5.50 secondes sauvegardée.
Frame à 6.97 secondes sauvegardée.
Frame à 1.40 secondes sauvegardée.
Frame à 8.17 secondes sauvegardée.
Frame à 10.00 secondes sauvegardée.
Frame à 16.10 secondes sauvegardée.
Error processing video 15_low1_3.mp4: could not convert string to float: '-'
Frame à 4.20 secondes sauvegardée.
Frame à 4.67 secondes sauvegardée.
Frame à 1.80 secondes sauvegardée.
Frame à 3.27 secondes sauvegardée.
Error processing video 14 controls countryside.mp4: could not convert string to float: '-'
Frame à 5.90 secondes sauvegardée

In [227]:
#Define uri for the frames
video_name = "15_low5_1" #without the .mp4 extension
frame_bucket_nb = 1 #1 or 2

frame_1_uri = f"gs://datapict{frame_bucket_nb}/{video_name}frame_1.jpg"
frame_2_uri = f"gs://datapict{frame_bucket_nb}/{video_name}frame_2.jpg"

In [228]:
#Test with the two following frames
prompt = """
These two images taken within seconds of each other are a first person view from inside a moving car, in a country where people drive on the left.
You are the driver of the camera's car, can you spot a situation that will end in an accident where another element is in the same trajectory as your car? If yes:
Say yes, describe the situation and give the elements involved.
If no:
Say no and describe what is shown in the video.
"""

'''
These two images taken within seconds of each other are a first person view from inside a moving car, in a country where people drive on the left.
You are the driver of the camera's car, can you spot a situation where an accident could happen with your car? If yes:
Describe the situation and give the elements involved.
'''
#This video is taken from inside a car, from the front passenger seat, in a country where people drive on the left. This video shows a situation where an accident is about to occur. Can you describe the situation shown in the video ?

frame_1 = Part.from_uri(frame_1_uri, mime_type="image/jpeg")
frame_2 = Part.from_uri(frame_2_uri, mime_type="image/jpeg")
contents = [frame_1, frame_2, prompt]

responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)

for response in responses:
    print(response.text, end="")

 Yes, there is a situation that will end in an accident. A bus is coming in the opposite direction and it is in the same trajectory as the car. The bus is on the left side of the road, and the car is on the right side of the road. If the bus continues to go straight, it will hit the car.